## Visualizer Raw Data or Pre-Processed Data

In [ ]:
from pcdet.datasets.kitti.kitti_dataset import *
from pcdet.datasets.dataset import *
from pcdet.datasets import DatasetTemplate
from pcdet.utils import common_utils, calibration_kitti, box_utils

import yaml
from easydict import EasyDict
from pathlib import Path
import pickle

from tools.visual_utils import open3d_vis_utils as Visualizer

dataset_cfg=EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/tools/cfgs/dataset_configs/kitti_dataset.yaml')))
class_names=['Car', 'Pedestrian', 'Cyclist']
file_path = '/home/rlab10/OpenPCDet/pcdet/datasets/kitti/kitti_dataset.py' 
# /home/rlab10/OpenPCDet
ROOT_DIR = (Path(file_path).resolve().parent / '../../../').resolve()
data_path = ROOT_DIR / 'data' / 'kitti'
ext = '.bin'
data = []
num_features = len(dataset_cfg.POINT_FEATURE_ENCODING.src_feature_list)

class DemoDataset(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=True, root_path=None, logger=None, data_processor_flag=False, fov_mode=False):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.split = self.dataset_cfg.DATA_SPLIT['train']
        self.root_split_path = self.root_path / ('training' if self.split != 'test' else 'testing')
        split_dir = self.root_path / 'ImageSets' / (self.split + '.txt')
        self.sample_id_list = [x.strip() for x in open(split_dir).readlines()] if split_dir.exists() else None
        self.demo_infos = []
        self.include_kitti_data(mode='train')
        self.data_processor_flag = data_processor_flag
        self.fov_mode = fov_mode

    def include_kitti_data(self, mode):
        if self.logger is not None:
            self.logger.info('DemoDataset: Loading raw KITTI dataset')
        demo_infos = []

        for info_path in self.dataset_cfg.INFO_PATH[mode]: # 'train', bc training=True
            info_path = self.root_path / info_path
            if not info_path.exists():
                continue
            # Read the data infos from kitti_infos_train.pkl
            with open(info_path, 'rb') as f:
                infos = pickle.load(f)
                demo_infos.extend(infos)
        # Add the newly loaded KITTI dataset information to kitti_infos list.
        self.demo_infos.extend(demo_infos)

    def __len__(self):
        return len(self.sample_id_list)
    
    def get_lidar(self, idx):
        lidar_file = self.root_split_path / 'velodyne' / ('%s.bin' % idx)
        assert lidar_file.exists()
        return np.fromfile(str(lidar_file), dtype=np.float32).reshape(-1, 4)

    def get_calib(self, idx):
        calib_file = self.root_split_path / 'calib' / ('%s.txt' % idx)
        assert calib_file.exists()
        return calibration_kitti.Calibration(calib_file)

    @staticmethod
    def get_fov_flag(pts_rect, img_shape, calib):
        """
        Args:
            pts_rect:
            img_shape:
            calib:

        Returns:

        """
        pts_img, pts_rect_depth = calib.rect_to_img(pts_rect)
        val_flag_1 = np.logical_and(pts_img[:, 0] >= 0, pts_img[:, 0] < img_shape[1])
        val_flag_2 = np.logical_and(pts_img[:, 1] >= 0, pts_img[:, 1] < img_shape[0])
        val_flag_merge = np.logical_and(val_flag_1, val_flag_2)
        pts_valid_flag = np.logical_and(val_flag_merge, pts_rect_depth >= 0)

        return pts_valid_flag

    def prepare_demo_data(self, data_dict):
        print('DemoDataset: prepare_demo_data() called')
        if 'gt_boxes' not in data_dict:
            assert 'gt_boxes' in data_dict, 'gt_boxes should be provided for demo visualization'
        else:         
            if data_dict.get('gt_boxes', None) is not None:
                selected = common_utils.keep_arrays_by_name(data_dict['gt_names'], self.class_names)
                data_dict['gt_boxes'] = data_dict['gt_boxes'][selected]
                data_dict['gt_names'] = data_dict['gt_names'][selected]
                gt_classes = np.array([self.class_names.index(n) + 1 for n in data_dict['gt_names']], dtype=np.int32)
                # already transformed 3D boxes LiDAR coord. + add number for the class
                gt_boxes = np.concatenate((data_dict['gt_boxes'], gt_classes.reshape(-1, 1).astype(np.float32)), axis=1)
                data_dict['gt_boxes'] = gt_boxes


            if data_dict.get('points', None) is not None:
                data_dict = self.point_feature_encoder.forward(data_dict)
            
            if self.data_processor_flag:
                print('DemoDataset: data processing activated')
                data_dict = self.data_processor.forward(data_dict=data_dict)
    
        return data_dict

    def __getitem__(self, index):
        print('DemoDataset: __getitem__ called')
        if self.merge_all_iters_to_one_epoch:
            index = index % len(self.demo_infos)
        # works
        info = copy.deepcopy(self.demo_infos[index])

        sample_idx = info['point_cloud']['lidar_idx']
        img_shape = info['image']['image_shape']
        calib = self.get_calib(sample_idx)
        get_item_list = self.dataset_cfg.get('GET_ITEM_LIST', ['points'])

        input_dict = {
            'frame_id': sample_idx,
        }

        if 'annos' in info:
            annos = info['annos']
            annos = common_utils.drop_info_with_name(annos, name = 'DontCare')
            loc, dims, rots = annos['location'], annos['dimensions'], annos['rotation_y']
            gt_names = annos['name']
            gt_boxes_camera = np.concatenate([loc, dims, rots[...,np.newaxis]], axis = 1).astype(np.float32)
            gt_boxes_lidar = box_utils.boxes3d_kitti_camera_to_lidar(gt_boxes_camera, calib)

        input_dict.update({
            'gt_names': gt_names,
            'gt_boxes': gt_boxes_lidar
        })

        if 'points' in get_item_list:
            points = self.get_lidar(sample_idx)

            if self.fov_mode:
                if self.dataset_cfg.FOV_POINTS_ONLY:
                    print('DemoDataset: fov mode activated')
                    # only the points in the FOV of the camera are important for me
                    pts_rect = calib.lidar_to_rect(points[:, 0:3])
                    fov_flag = self.get_fov_flag(pts_rect, img_shape, calib)
                    points = points[fov_flag]
            input_dict['points'] = points

        data_dict = self.prepare_demo_data(data_dict=input_dict)

        return data_dict

def main(mode=str):
    logger = common_utils.create_logger()
    if mode == 'raw':
        logger.info('-----------------Quick Visualizer Demo of raw data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        demo_dataset = DemoDataset(dataset_cfg=dataset_cfg, class_names=class_names, training=False, root_path=data_path, logger=logger, data_processor_flag=False, fov_mode=False)
        logger.info(f'Total number of samples: \t{len(demo_dataset)}')
        # raw data not  0-4
        #demo_dataset.demo_infos[4]
        #demo_dataset[0]
        # get_infos oder prepare_data hier implementieren(flow: include_kitti_data->__getitem__->prepare_data)
        # or using KittiDataset for accessing the
        data_dict = demo_dataset[0]
        print('frame_id: ', data_dict['frame_id'])
        points = data_dict['points']
        gt_boxes = data_dict['gt_boxes']

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, point_colors=None, draw_origin=True)
        logger.info('Demo visualization of raw data done.')

    elif mode == 'processed':
        logger.info('-----------------Quick Visualizer Demo of pre-processed data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        demo_dataset = DemoDataset(dataset_cfg=dataset_cfg, class_names=class_names, training=True, root_path=data_path, logger=logger, data_processor_flag=True, fov_mode=True)
        logger.info(f'Total number of samples: \t{len(demo_dataset)}')
       
        data_dict = demo_dataset[0]
        print('frame_id: ', data_dict['frame_id'])
        points = data_dict['points']
        gt_boxes = data_dict['gt_boxes']

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, point_colors=None, draw_origin=True)
        logger.info('Demo visualization of pre-processed data done.')

    elif mode == 'val':
        logger.info('-----------------Quick Visualizer Demo of validation .pkl data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        pkl_path = ROOT_DIR / 'data' / 'kitti' / 'data_test_pipeline' / 'visualizer' / 'kitti_val_dataset.pkl'

        with open(pkl_path, 'rb') as f:
            demo_val_dataset = pickle.load(f)

        data_dict = demo_val_dataset[0]
        points = data_dict['points']
        gt_boxes_lidar = data_dict['annos']['gt_boxes_lidar']
        #lidar_idx = data_dict['point_cloud']['lidar_idx']

        #class_n = data_dict['annos']['name']
        #print(f"\nLidar IDX: {lidar_idx}")
        #print(f"Points: {points}\n")
        #print(f"Class Names: {class_n}\n")
        #print(f"Ground Truth Boxes (LiDAR): {gt_boxes_lidar}")

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes_lidar, point_colors=None, draw_origin=True, vc='val')
        logger.info('Demo visualization of pre-processed validation data done.')

    elif mode == 'train':
        logger.info('-----------------Quick Visualizer Demo of training .pkl data-------------------------')
        logger.info(f'Mode for visualization: {mode}')
        #pkl_path = ROOT_DIR / 'data' / 'kitti' / 'data_test_pipeline' / 'visualizer' / 'kitti_train_dataset.pkl'
        pkl_path = ROOT_DIR / 'data' / 'kitti' / 'kitti_train_dataset.pkl'
    
        with open(pkl_path, 'rb') as f:
            demo_train_dataset = pickle.load(f)

        data_list = demo_train_dataset[0] # specific sample from row no. in train.txt, e.g. (34-1) - 000067)
        # [0] - original; [1] - gt_sampling; [2] - flip_x; [3] - w_rotation; [4] - l_rotation

        try:
            print("\nPlease select an option for the visualization:")
            print('[0] Original')
            print('[1] Ground Truth Sampling (gt_sampling)')
            print('[2] Flip along X-axis (flip_x)')
            print('[3] World rotation (w_rotation)')
            print('[4] Local rotation (l_rotation)\n')
            user_choice = int(input('Your selection: '))
            if user_choice not in range(5):
                raise ValueError('Invalid selection')
        except ValueError as e:
            print(f"Error: {e}. Please enter a number between 0 and 4.")
            logger.error("Invalid user input for the visualization mode.")
            return
        
        switch_cases = {0: "original", 1: "gt_sampling", 2: "flip_x", 3: "w_rotation", 4: "l_rotation"}
        
        selected_data = data_list[user_choice]
        points = selected_data['points']
        gt_boxes = selected_data['gt_boxes']

        if user_choice in [3, 4]: 
            noise_key = 'noise_glob_rot' if user_choice == 3 else 'noise_loc_rot'
            noise_rot = selected_data.get(noise_key, None)
            if noise_rot is not None:
                logger.info(f'Noise rotation for {switch_cases[user_choice]}: {noise_rot}')

        Visualizer.draw_demo_scenes(points=points, gt_boxes=gt_boxes, point_colors=None, draw_origin=True)
        logger.info('Demo visualization of training data done.')

if __name__ == '__main__':
    #main(mode='raw')
    #main(mode='processed')
    #main(mode='val')
    main(mode='train')

In [ ]:
import open3d
import numpy as np

def text_3d(text, pos, direction=None, degree=0.0, density=10, font='/usr/share/fonts/truetype/freefont/FreeMono.ttf', font_size=10):
    if direction is None:
        direction = (0., 0., 1.) # default positioned to z-axis

    from PIL import Image, ImageFont, ImageDraw
    from pyquaternion import Quaternion
   

    # Adjust font size based on density
    font_obj = ImageFont.truetype(font, font_size * density)
    #font_obj = ImageFont.truetype(font, font_size)
    left, top, right, bottom = font_obj.getbbox(text)
    width = right - left
    height = bottom - top
    font_dim = (width, height)
    

    # Create an image with the text
    img = Image.new('RGB', font_dim, color=(255, 255, 255))
    draw = ImageDraw.Draw(img) # white background image
    draw.text((-35, -35), text, font=font_obj, fill=(0, 0, 0)) # black
    img = np.asarray(img)

    # Create a mask for the text pixels
    img_mask = img[:, :, 0] < 128
    indices = np.indices([*img.shape[0:2], 1])[:, img_mask, 0].reshape(3, -1).T

    # Create a point cloud and adjust point density
    pcd = open3d.geometry.PointCloud()
    pcd.colors = open3d.utility.Vector3dVector(img[img_mask, :].astype(float) / 255.0)
    pcd.points = open3d.utility.Vector3dVector(indices / (1000 * density))

    # Rotate and translate the point cloud
    raxis = np.cross([0.0, 0.0, 1.0], direction)
    if np.linalg.norm(raxis) < 1e-6:
        raxis = (0.0, 0.0, 1.0)
    trans = (Quaternion(axis=raxis, radians=np.arccos(direction[2])) *
             Quaternion(axis=direction, degrees=degree)).transformation_matrix
    trans[0:3, 3] = np.asarray(pos)
    pcd.transform(trans)

    return pcd

rh_system = open3d.geometry.TriangleMesh.create_coordinate_frame(size=0.02, origin=[0, 0, 0])
pcd_10 = text_3d('1 - Car', pos=[0, 0, 0.02], font_size=10, density=5)
#open3d.visualization.draw_geometries([pcd_10, chessboard_coord])
vis = open3d.visualization.Visualizer()
vis.create_window(width=640, height=480)
vis.add_geometry(pcd_10)
vis.add_geometry(rh_system)
vis.run()
vis.destroy_window()

In [ ]:
import open3d as o3d
import numpy as np

def create_text_points(text="0", scale=0.1, density=800):  # Sehr hohe Punktdichte für eine durchgehende Linie
    """
    Erstellt eine Punktwolke in Form eines Texts
    """
    if text == "0":
        # Eine einzelne, sehr dichte Punktreihe für die "0"
        t = np.linspace(0, 2*np.pi, density)
        # Schmaler und höher für einen "I"-ähnlichen Look
        x = 0.25 * np.cos(t) * scale  # Reduzierte Breite
        y = np.sin(t) * scale  # Normale Höhe
        z = np.zeros_like(t)
        points = np.column_stack((x, y, z))
    else:
        points = np.array([[0, 0, 0]])
    
    return points

def visualize_bbox_with_label():
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=960, height=540)
    
    
    bbox = o3d.geometry.OrientedBoundingBox(
        center=[0, 0, 0],
        R=np.eye(3),
        extent=[1, 1, 1]
    )
    bbox.color = [1, 0, 0]
    vis.add_geometry(bbox)
    
    points = np.random.rand(1000, 3) * 0.8
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.paint_uniform_color([0, 0.7, 0])
    vis.add_geometry(pcd)
    
    label_position = bbox.get_center() + bbox.extent/2 * np.array([-1, 1, 1])
    text_points = create_text_points(scale=0.1) + label_position + np.array([0.1, 0, 0])
    
    label_pcd = o3d.geometry.PointCloud()
    label_pcd.points = o3d.utility.Vector3dVector(text_points)
    label_pcd.paint_uniform_color([1, 1, 1])
    
    # Minimale Tiefe für kaum sichtbaren 3D-Effekt
    depth = 0.005  # Stark reduzierte Tiefe
    
    # Minimale Zwischenpunkte
    steps = 2
    line_points = []
    line_indices = []
    
    for i, point in enumerate(text_points):
        for step in range(steps):
            depth_step = depth * (step / (steps - 1))
            current_point = point + np.array([0, 0, -depth_step])
            line_points.append(current_point)
            if step > 0:
                line_indices.append([i*steps + step - 1, i*steps + step])
    
    lines = o3d.geometry.LineSet()
    lines.points = o3d.utility.Vector3dVector(line_points)
    lines.lines = o3d.utility.Vector2iVector(line_indices)
    lines.paint_uniform_color([1, 1, 1])
    
    vis.add_geometry(label_pcd)
    vis.add_geometry(lines)
    
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.5, 0.5, 0.5])
    opt.point_size = 1.0  # Sehr kleine Punktgröße für eine feine Linie
    
    vis.run()
    vis.destroy_window()

if __name__ == "__main__":
    visualize_bbox_with_label()

In [ ]:
import open3d
import numpy as np
import open3d.visualization

# Beispiel-Punktwolke erstellen
points = np.random.rand(1000, 3)  # Zufällige Punkte für die Demonstration
point_cloud = open3d.geometry.PointCloud()
point_cloud.points = open3d.utility.Vector3dVector(points)

# Erstellen einer Bounding Box für den Punktwolkenbereich
bounding_box = open3d.geometry.OrientedBoundingBox.create_from_points(point_cloud.points)
bounding_box.color = (1, 0, 0)  # Farbe der Box setzen

# Visualisierung mit O3DVisualizer, wie in einem Jupyter Notebook
app = open3d.visualization.gui.Application.instance
app.initialize()
vis = open3d.visualization.O3DVisualizer("Open3D - 3D Text", 1024, 768)
vis.show_settings = True
vis.add_geometry("Points", point_cloud)
vis.add_geometry("Bounding Box", bounding_box)
vis.show()
#app.add_window()
#app.run()


